In [2]:
import pandas as pd            #读取数据
import torch
from torch import nn		   #继承nn.Module构建rnn网络
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader #构建迭代器
import matplotlib.pyplot as plt			#绘图
import h3

In [4]:
# 读取数据
train_power_forecast_history = pd.read_csv('./训练集/power_forecast_history.csv')
train_power = pd.read_csv('./训练集/power.csv')
train_stub_info = pd.read_csv('./训练集/stub_info.csv')

test_power_forecast_history = pd.read_csv('./测试集/power_forecast_history.csv')
test_stub_info = pd.read_csv('./测试集/stub_info.csv')
# 聚合数据
train_df = train_power_forecast_history.groupby(['id_encode','ds']).head(1)
del train_df['hour']

test_df = test_power_forecast_history.groupby(['id_encode','ds']).head(1)
del test_df['hour']

tmp_df = train_power.groupby(['id_encode','ds'])['power'].sum()
tmp_df.columns = ['id_encode','ds','power']

# 合并充电量数据
train_df = train_df.merge(tmp_df, on=['id_encode','ds'], how='left')

### 合并数据
train_df = train_df.merge(train_stub_info, on='id_encode', how='left')
test_df = test_df.merge(test_stub_info, on='id_encode', how='left')
train_df['flag'] = train_df['flag'].map({'A':0,'B':1})
test_df['flag'] = test_df['flag'].map({'A':0,'B':1})

In [6]:
train_df['latitude'] = train_df['h3'].apply(lambda x: h3.h3_to_geo(x)[0])
train_df['longitude'] = train_df['h3'].apply(lambda x: h3.h3_to_geo(x)[1])
test_df['latitude'] = test_df['h3'].apply(lambda x: h3.h3_to_geo(x)[0])
test_df['longitude'] = test_df['h3'].apply(lambda x: h3.h3_to_geo(x)[1])
train_df=train_df.drop('h3', axis=1)
test_df=test_df.drop('h3', axis=1)

In [7]:
train_df.head()

,id_encode,ele_price,ser_price,after_ser_price,total_price,f1,f2,f3,ds,power,parking_free,flag,ac_equipment_kw,dc_equipment_kw,latitude,longitude
0,0,0.64,0.95,0.31,1.59,0.0,0.0,1.0,20220415,2288.2240,1.0,0,0.0,1440.0,31.523294,120.096637
1,0,0.64,0.95,0.31,1.59,0.0,0.0,1.0,20220416,2398.5730,1.0,0,0.0,1440.0,31.523294,120.096637
2,0,0.64,0.95,0.31,1.59,0.0,0.0,1.0,20220417,2313.0330,1.0,0,0.0,1440.0,31.523294,120.096637
3,0,0.64,0.95,0.31,1.59,0.0,0.0,1.0,20220418,2095.3259,1.0,0,0.0,1440.0,31.523294,120.096637
4,0,0.64,0.95,0.31,1.59,0.0,0.0,1.0,20220419,1834.3590,1.0,0,0.0,1440.0,31.523294,120.096637


In [8]:
test_df.head()

,id_encode,ele_price,ser_price,after_ser_price,total_price,f1,f2,f3,ds,parking_free,flag,ac_equipment_kw,dc_equipment_kw,latitude,longitude
0,0,0.6414,0.3086,0.3086,0.95,2,178.0,NaN,20230415,1.0,0,0.0,1440.0,31.523294,120.096637
1,0,0.6414,0.3086,0.3086,0.95,2,178.0,NaN,20230416,1.0,0,0.0,1440.0,31.523294,120.096637
2,0,0.6414,0.3086,0.3086,0.95,2,178.0,NaN,20230417,1.0,0,0.0,1440.0,31.523294,120.096637
3,0,0.6414,0.3086,0.3086,0.95,2,178.0,NaN,20230418,1.0,0,0.0,1440.0,31.523294,120.096637
4,0,0.6414,0.3086,0.3086,0.95,2,178.0,NaN,20230419,1.0,0,0.0,1440.0,31.523294,120.096637


In [9]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [10]:
# convert the dataframe to tensor again
data_tensor = torch.tensor(train_df.values)

# Extract the id_encode column as a separate tensor
id_encode_tensor = data_tensor[:, 0].int()

# Create a dictionary to hold tensors for each id_encode
tensor_dict = {}

# Unique id_encodes
unique_ids = torch.unique(id_encode_tensor).numpy()

# Loop over unique id_encodes and create tensors
for uid in unique_ids:
    mask = (id_encode_tensor == uid)
    tensor_dict[uid] = data_tensor[mask]

# Now let's stack these tensors to create a 3D tensor
# First, we need to pad each tensor to have the same shape
max_length = max([t.shape[0] for t in tensor_dict.values()])
padded_tensors = []

for t in tensor_dict.values():
    # Pad tensor
    pad_len = max_length - t.shape[0]
    padded_tensor = nn.functional.pad(t, pad=(0, 0, 0, pad_len))
    padded_tensors.append(padded_tensor)

# Stack tensors to create a 3D tensor
final_tensor = torch.stack(padded_tensors)

final_tensor.shape

torch.Size([500, 365, 16])

看下0站点，第二天的数据

In [11]:
final_tensor[0,1]

tensor([0.0000e+00, 6.4000e-01, 9.5000e-01, 3.1000e-01, 1.5900e+00, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 2.0220e+07, 2.3986e+03, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 1.4400e+03, 3.1523e+01, 1.2010e+02], dtype=torch.float64)

In [12]:
# check final_tensor 
has_nan = torch.any(torch.isnan(final_tensor))
print(has_nan)

tensor(False)


In [13]:
# Identify the index of the 'power' column in the original dataframe
power_index = train_df.columns.get_loc('power')

# Split X and y based on the correct index
X = torch.cat((final_tensor[:, :, :power_index], final_tensor[:, :, power_index+1:]), dim=2)
y = final_tensor[:, :, power_index]

X.shape, y.shape

(torch.Size([500, 365, 15]), torch.Size([500, 365]))

In [14]:
from torch.utils.data import DataLoader, random_split

num_stations = X.shape[0]
num_train_stations = int(0.8 * num_stations)
num_val_stations = num_stations - num_train_stations
# Create a dataset with stations as the main data points
station_dataset = TensorDataset(X, y)

# Split the dataset by stations using random_split
train_dataset, val_dataset = random_split(station_dataset, [num_train_stations, num_val_stations])

# Extract X and y from the datasets
X_train, y_train = train_dataset[:][0], train_dataset[:][1]
X_val, y_val = val_dataset[:][0], val_dataset[:][1]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

(torch.Size([400, 365, 15]),
 torch.Size([400, 365]),
 torch.Size([100, 365, 15]),
 torch.Size([100, 365]))

In [15]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
#check the shapes of batches
for batch_x, batch_y in train_loader:
    print(batch_x.shape)
    print(batch_y.shape)
    break

torch.Size([32, 365, 15])
torch.Size([32, 365])


In [18]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1):
        super(RNNModel, self).__init__()

        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # Output a prediction for each day

    def forward(self, x):
        # RNN layers
        out, _ = self.rnn(x)

        # Output layer
        out = self.fc(out)
        out = out.squeeze(-1)  # Remove the last dimension to get shape as (batch_size, sequence_length)
        return out

# Create the flexible model
model = RNNModel(input_dim=15, hidden_dim=256)
model

RNNModel(
  (rnn): RNN(15, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [20]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Define the RMSE loss function
criterion = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [21]:
# Set number of epochs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 1000

# Training loop with the modified model
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training mode
    model.train()
    # Move model and data to GPU
    model = model.to(device)

    # Training pass
    train_loss = 0.0
    for batch_x, batch_y in DataLoader(train_dataset, batch_size=32, shuffle=True):
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()
        predictions = model(batch_x.float())
        loss = criterion(predictions, batch_y.float())
        #print(batch_x.shape,predictions.shape,batch_y.float().shape)
        #break
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss / len(train_dataset))
    #break
    # Evaluation mode
    model.eval()
    # Move model and data to GPU
    model = model.to(device)

    # Validation pass
    val_loss = 0.0
    with torch.no_grad():
        for batch_x, batch_y in DataLoader(val_dataset, batch_size=32):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            predictions = model(batch_x.float())
            loss = criterion(predictions, batch_y.float())
            val_loss += loss.item()

    val_losses.append(val_loss / len(val_dataset))

    # Print epoch results
    if epoch % 500 == 499:
      print(f"Epoch {epoch+1}/{num_epochs} => "
          f"Train loss: {train_loss / len(train_dataset):.4f}, "
          f"Validation loss: {val_loss / len(val_dataset):.4f}")
    #print(f"Epoch {epoch+1}/{num_epochs} => "
          #f"Train loss: {train_loss / len(train_dataset):.4f}, "
          #f"Validation loss: {val_loss / len(val_dataset):.4f}")

#train_losses, val_losses

Epoch 500/1000 => Train loss: 38.6352, Validation loss: 49.4562
Epoch 1000/1000 => Train loss: 39.7252, Validation loss: 49.5566


In [22]:
test_tensor = torch.tensor(test_df.values)
test_tensor.shape

torch.Size([3500, 15])

In [23]:
# Extract the id_encode column as a separate tensor
id_encode_tensor = test_tensor[:, 0].int()

# Create a dictionary to hold tensors for each id_encode
tensor_dict = {}

# Unique id_encodes
unique_ids = torch.unique(id_encode_tensor).numpy()

# Loop over unique id_encodes and create tensors
for uid in unique_ids:
    mask = (id_encode_tensor == uid)
    tensor_dict[uid] = test_tensor[mask]

# Now let's stack these tensors to create a 3D tensor
# First, we need to pad each tensor to have the same shape
max_length = max([t.shape[0] for t in tensor_dict.values()])
padded_tensors = []

for t in tensor_dict.values():
    # Pad tensor
    pad_len = max_length - t.shape[0]
    padded_tensor = nn.functional.pad(t, pad=(0, 0, 0, pad_len))
    padded_tensors.append(padded_tensor)

# Stack tensors to create a 3D tensor
test_tensor = torch.stack(padded_tensors)

test_tensor.shape

torch.Size([500, 7, 15])

In [24]:
# Evaluation mode
model.eval()
# Move model and data to GPU
model = model.to(device)
predictions = model(test_tensor.to(device).float())
predictions.shape

torch.Size([500, 7])

In [28]:
predictions=predictions.reshape(-1)
test_df['power']=predictions..detach().to('cpu').numpy()
test_df['power'] = test_df['power'].apply(lambda x: 0 if x<0 else x)
test_df[['id_encode','ds','power']].to_csv('resultRNN.csv', index=False)

SyntaxError: invalid syntax (3014765363.py, line 2)